# Basic Feature Engineering in Keras 

**Learning Objectives**


1. Create an input pipeline using tf.data
2. Engineer features to create categorical, crossed, and numerical feature columns




## Overview 
In this lab, we utilize feature engineering to improve the prediction of housing prices using a Keras Sequential Model.  

In [1]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import tensorflow
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import feature_column as fc
from tensorflow.keras import layers

print("TensorFlow version: ", tf.version.VERSION)

2025-05-14 00:03:29.597137: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version:  2.12.0


Many of the Google Machine Learning Courses Programming Exercises use the  [California Housing Dataset](https://developers.google.com/machine-learning/crash-course/california-housing-data-description
), which contains data drawn from the 1990 U.S. Census.  Our lab dataset has been pre-processed so that there are no missing values.

In [2]:
!ls -l ../data/

total 2220
-rw-r--r-- 1 jupyter jupyter 1435071 May 12 07:39 housing_pre-proc.csv
-rw-r--r-- 1 jupyter jupyter  123590 May 12 07:39 taxi-test.csv
-rw-r--r-- 1 jupyter jupyter  579055 May 12 07:39 taxi-train.csv
-rw-r--r-- 1 jupyter jupyter  123114 May 12 07:39 taxi-valid.csv


Let's read in the dataset and create a Pandas dataframe.

In [3]:
housing_df = pd.read_csv("../data/housing_pre-proc.csv", on_bad_lines="skip")
housing_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


We can use .describe() to see some summary statistics for the numeric fields in our dataframe. Note, for example, the count row and corresponding columns. The count shows 20433.000000 for all feature columns. Thus, there are no missing values.

In [4]:
housing_df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


####  Split the dataset for ML

The dataset we loaded was a single CSV file. We will split this into train, validation, and test sets.


In [5]:
train, test = train_test_split(housing_df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

print(len(train), "train examples")
print(len(val), "validation examples")
print(len(test), "test examples")

13076 train examples
3270 validation examples
4087 test examples


Now, we need to output the split files.  We will specifically need the test.csv later for testing.  You should see the files appear in the home directory.


In [6]:
train.to_csv("../data/housing-train.csv", encoding="utf-8", index=False)

In [7]:
val.to_csv("../data/housing-val.csv", encoding="utf-8", index=False)

In [8]:
test.to_csv("../data/housing-test.csv", encoding="utf-8", index=False)

In [9]:
!head ../data/housing*.csv

==> ../data/housing-test.csv <==
longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
-122.07,37.71,40.0,1808.0,302.0,746.0,270.0,5.3015,254900.0,NEAR BAY
-117.97,33.81,30.0,2406.0,462.0,1753.0,456.0,4.485,180600.0,<1H OCEAN
-118.1,33.93,33.0,1474.0,325.0,1205.0,335.0,3.1397,166800.0,<1H OCEAN
-119.75,36.72,22.0,3247.0,859.0,4179.0,881.0,1.3343,60800.0,INLAND
-119.72,34.44,39.0,1489.0,304.0,700.0,268.0,3.8819,289900.0,<1H OCEAN
-121.78,39.71,8.0,140.0,28.0,84.0,29.0,2.125,179200.0,INLAND
-117.4,34.0,31.0,1192.0,307.0,1013.0,283.0,2.0742,76200.0,INLAND
-122.29,37.87,52.0,895.0,198.0,386.0,204.0,3.875,182600.0,NEAR BAY
-121.65,39.76,31.0,1599.0,318.0,794.0,303.0,3.0,96700.0,INLAND

==> ../data/housing-train.csv <==
longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
-117.24,34.12,29.0,2654.0,667.0,1822.0,593.0,2.1563,72300.

## Create an input pipeline using tf.data

Next, we will wrap the dataframes with [tf.data](https://www.tensorflow.org/guide/datasets). This will enable us  to use feature columns as a bridge to map from the columns in the Pandas dataframe to features used to train the model. 

**Exercise.** Here, we create an input pipeline using tf.data.  This function is missing two lines.  Correct and run the cell.

In [21]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()

    ds = tf.data.Dataset.from_tensor_slices(dict(dataframe))

    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

Next we initialize the training and validation datasets.

In [22]:
batch_size = 32
train_ds = df_to_dataset(train)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)

**Exercise.** Now that we have created the input pipeline, let's call it to see the format of the data it returns. We have used a small batch size to keep the output readable.

In [23]:
for feature_batch, label_batch in train_ds.take(1):
    print("Every feature:", list(feature_batch.keys()))
    print(
        "A batch of households:",
        # TODO: Your code goes here
    )
    print(
        "A batch of ocean_proximity:",
        # TODO: Your code goes here
    )
    print(
        "A batch of targets:",
        # TODO: Your code goes here
    )

2025-05-14 00:10:59.814673: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype double and shape [13076]
	 [[{{node Placeholder/_5}}]]
2025-05-14 00:10:59.815431: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [13076]
	 [[{{node Placeholder/_0}}]]


ValueError: too many values to unpack (expected 2)

We can see that the dataset returns a dictionary of column names (from the dataframe) that map to column values from rows in the dataframe.

#### Numeric columns
The output of a feature column becomes the input to the model. A numeric is the simplest type of column. It is used to represent real valued features. When using this column, your model will receive the column value from the dataframe unchanged.

**Exercise.** In the California housing prices dataset, most columns from the dataframe are numeric.  Let' create a variable called `numeric_cols` to hold only the numerical feature columns.

In [ ]:
numeric_cols = # TODO: Your code goes here

#### Max-min scaler function
It is very important for numerical variables to get scaled before they are "fed" into the neural network. Here we use min-max scaling. Here we are creating a function named 'get_scal' which takes a list of numerical features and returns a 'minmax' function, which will be used in `tf.feature_column.numeric_column()` as normalizer_fn in parameters. 'Minmax' function itself takes a 'numerical' number from a particular feature and return scaled value of that number. 

**Exercise.** Next, we scale the numerical feature columns that we assigned to the variable "numeric cols".

In [ ]:
# Scalar def get_scal(feature):
def get_scal(feature):
    def minmax(x):
        mini = # TODO: Your code goes here
        maxi = # TODO: Your code goes here
        return # TODO: Your code goes here
    return(minmax)

In [ ]:
feature_columns = []
for header in numeric_cols:
    scal_input_fn = get_scal(
        # TODO: Your code goes here
    )
    feature_columns.append(
        fc.numeric_column(
            # TODO: Your code goes here
        )
    )

Next, we should validate the total number of feature columns.  Compare this number to the number of numeric features you input earlier.

In [ ]:
print("Total number of feature columns: ", len(feature_columns))

### Using the Keras Sequential Model

Next, we will run this cell to compile and fit the Keras Sequential model.

In [ ]:
# Model create
feature_layer = tf.keras.layers.DenseFeatures(feature_columns, dtype="float64")

model = tf.keras.Sequential(
    [
        feature_layer,
        layers.Dense(12, activation="relu"),
        layers.Dense(8, activation="relu"),
        layers.Dense(1, activation="linear", name="median_house_value"),
    ]
)

# Model compile
model.compile(optimizer="adam", loss="mse", metrics=["mse"])

# Model Fit
history = model.fit(train_ds, validation_data=val_ds, epochs=32)

Next we show loss as  Mean Square Error (MSE).  Remember that MSE is the most commonly used regression loss function. MSE is the sum of squared distances between our target variable (e.g. housing median age) and predicted values.

In [ ]:
loss, mse = model.evaluate(train_ds)
print("Mean Squared Error", mse)

#### Visualize the model loss curve

Next, we will use matplotlib to draw the model's loss curves for training and validation.  A line plot is also created showing the mean squared error loss over the training epochs for both the train (blue) and test (orange) sets.

In [ ]:
def plot_curves(history, metrics):
    nrows = 1
    ncols = 2
    fig = plt.figure(figsize=(10, 5))

    for idx, key in enumerate(metrics):
        ax = fig.add_subplot(nrows, ncols, idx + 1)
        plt.plot(history.history[key])
        plt.plot(history.history[f"val_{key}"])
        plt.title(f"model {key}")
        plt.ylabel(key)
        plt.xlabel("epoch")
        plt.legend(["train", "validation"], loc="upper left");

In [ ]:
plot_curves(history, ["loss", "mse"])

### Load test data

Next, we read in the test.csv file and validate that there are no null values.  

Again, we can use .describe() to see some summary statistics for the numeric fields in our dataframe.  The count shows 4087.000000 for all feature columns. Thus, there are no missing values.

In [ ]:
test_data = pd.read_csv("../data/housing-test.csv")
test_data.describe()

**Exercise.** Now that we have created an input pipeline using tf.data and compiled a Keras Sequential Model, we now create the input function for the test data and to initialize the `test_predict` variable.

In [ ]:
def test_input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(
        # TODO: Your code goes here
    )

In [ ]:
test_predict = test_input_fn(dict(test_data))

#### Prediction:  Linear Regression

Before we begin to feature engineer our feature columns, we should predict the median house value.  By predicting the median house value now, we can then compare it with the median house value after feature engineeing.

To predict with Keras, you simply call [model.predict()](https://keras.io/models/model/#predict) and pass in the housing features you want to predict the median_house_value for. Note:  We are predicting the model locally.

In [ ]:
predicted_median_house_value = model.predict(test_predict)

Next, we run two predictions in separate cells - one where ocean_proximity=INLAND and one where ocean_proximity= NEAR OCEAN. 

In [ ]:
# Ocean_proximity is INLAND
model.predict(
    {
        "longitude": tf.convert_to_tensor([-121.86]),
        "latitude": tf.convert_to_tensor([39.78]),
        "housing_median_age": tf.convert_to_tensor([12.0]),
        "total_rooms": tf.convert_to_tensor([7653.0]),
        "total_bedrooms": tf.convert_to_tensor([1578.0]),
        "population": tf.convert_to_tensor([3628.0]),
        "households": tf.convert_to_tensor([1494.0]),
        "median_income": tf.convert_to_tensor([3.0905]),
        "ocean_proximity": tf.convert_to_tensor(["INLAND"]),
    },
    steps=1,
)

In [ ]:
# Ocean_proximity is NEAR OCEAN
model.predict(
    {
        "longitude": tf.convert_to_tensor([-122.43]),
        "latitude": tf.convert_to_tensor([37.63]),
        "housing_median_age": tf.convert_to_tensor([34.0]),
        "total_rooms": tf.convert_to_tensor([4135.0]),
        "total_bedrooms": tf.convert_to_tensor([687.0]),
        "population": tf.convert_to_tensor([2154.0]),
        "households": tf.convert_to_tensor([742.0]),
        "median_income": tf.convert_to_tensor([4.9732]),
        "ocean_proximity": tf.convert_to_tensor(["NEAR OCEAN"]),
    },
    steps=1,
)

The arrays returns a predicted value.  What do these numbers mean?  Let's compare this value to the test set.  

Go to the test.csv you read in a few cells up.  Locate the first line and find the median_house_value - which should be 249,000 dollars near the ocean. What value did your model predicted for the median_house_value? Was it a solid model performance? Let's see if we can improve this a bit with feature engineering!  


## Engineer features to create categorical and numerical features

**Exercise.**  Create a cell that indicates which features will be used in the model.  
Note:  Be sure to bucketize 'housing_median_age' and ensure that 'ocean_proximity' is one-hot encoded.  And, don't forget your numeric values!

In [ ]:
numeric_cols = # TODO: Your code goes here

bucketized_cols = # TODO: Your code goes here

# indicator columns,Categorical features
categorical_cols = # TODO: Your code goes here

Next, we scale the numerical, bucktized, and categorical feature columns that we assigned to the variables in the preceding cell.

In [ ]:
# Scalar def get_scal(feature):
def get_scal(feature):
    def minmax(x):
        mini = train[feature].min()
        maxi = train[feature].max()
        return (x - mini) / (maxi - mini)

    return minmax

In [ ]:
# All numerical features - scaling
feature_columns = []
for header in numeric_cols:
    scal_input_fn = get_scal(header)
    feature_columns.append(
        fc.numeric_column(header, normalizer_fn=scal_input_fn)
    )

### Categorical Feature
In this dataset, 'ocean_proximity' is represented as a string.  We cannot feed strings directly to a model. Instead, we must first map them to numeric values. The categorical vocabulary columns provide a way to represent strings as a one-hot vector.

**Exercise.** Next, we create a categorical feature using `ocean_proximity`.

In [ ]:
for feature_name in categorical_cols:
    vocabulary = # TODO: Your code goes here
    categorical_c = # TODO: Your code goes here
    one_hot = # TODO: Your code goes here
    
    feature_columns.append(one_hot)

### Bucketized Feature

Often, you don't want to feed a number directly into the model, but instead split its value into different categories based on numerical ranges. Consider our raw data that represents a homes' age. Instead of representing the house age as a numeric column, we could split the home age into several buckets using a [bucketized column](https://www.tensorflow.org/api_docs/python/tf/feature_column/bucketized_column). Notice the one-hot values below describe which age range each row matches.

Next we create a bucketized column using 'housing_median_age'


In [ ]:
age = fc.numeric_column("housing_median_age")

# Bucketized cols
age_buckets = # TODO: Your code goes here

feature_columns.append(age_buckets)

### Feature Cross

Combining features into a single feature, better known as [feature crosses](https://developers.google.com/machine-learning/glossary/#feature_cross), enables a model to learn separate weights for each combination of features.

**Exercise.** Next, we create a feature cross of 'housing_median_age' and 'ocean_proximity'.

In [ ]:
vocabulary = housing_df['ocean_proximity'].unique()
ocean_proximity = fc.categorical_column_with_vocabulary_list('ocean_proximity',
                                                             vocabulary)

crossed_feature = # TODO: Your code goes here
crossed_feature = fc.indicator_column(crossed_feature)

feature_columns.append(crossed_feature)

Next, we should validate the total number of feature columns.  Compare this number to the number of numeric features you input earlier.

In [ ]:
print("Total number of feature columns: ", len(feature_columns))

Next, we will run this cell to compile and fit the Keras Sequential model.  This is the same model we ran earlier.

In [ ]:
# Model create
feature_layer = tf.keras.layers.DenseFeatures(feature_columns, dtype="float64")

model = tf.keras.Sequential(
    [
        feature_layer,
        layers.Dense(12, activation="relu"),
        layers.Dense(8, activation="relu"),
        layers.Dense(1, activation="linear", name="median_house_value"),
    ]
)

# Model compile
model.compile(optimizer="adam", loss="mse", metrics=["mse"])

# Model Fit
history = model.fit(train_ds, validation_data=val_ds, epochs=32)

Next, we show loss and mean squared error then plot the model.

In [ ]:
loss, mse = model.evaluate(train_ds)
print("Mean Squared Error", mse)

In [ ]:
plot_curves(history, ["loss", "mse"])

**Exercise.** Get a prediction from the model.  Note:  You may use the same values from the previous prediciton.  

In [ ]:
model.predict({
    'longitude': # TODO: Your code goes here,
    'latitude': # TODO: Your code goes here,
    'housing_median_age': # TODO: Your code goes here,
    'total_rooms': # TODO: Your code goes here,
    'total_bedrooms': # TODO: Your code goes here,
    'population': # TODO: Your code goes here,
    'households': # TODO: Your code goes here,
    'median_income': # TODO: Your code goes here,
    'ocean_proximity': # TODO: Your code goes here)
}, steps=1)

### Analysis 

The array returns a predicted value.  Compare this value to the test set you ran earlier. Your predicted value may be a bit better.

Now that you have your "feature engineering template" setup, you can experiment by creating additional features.  For example, you can create derived features, such as households per population, and see how they impact the model.  You can also experiment with replacing the features you used to create the feature cross.
 

Copyright 2021 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.